# Using XGBoosting with Preprocessed Features

## Importing data and creating train, val, test splits

### Importing preprocessed data

In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer

from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

#Reading Raw X_train
X_train = pd.read_csv('/Users/niki/code/Niki827/watt_squad/raw_data/train.csv')
X_test = pd.read_csv('/Users/niki/code/Niki827/watt_squad/raw_data/test.csv')

#Log columns
f_logs = [
    'precip_1h:mm',
    'prob_precip_1h:p',
    'clear_sky_rad:W',
    'clear_sky_energy_1h:J',
    'diffuse_rad:W',
    'diffuse_rad_1h:Wh',
    'direct_rad:W',
    'direct_rad_1h:Wh',
    'global_rad:W',
    'global_rad_1h:Wh',
    'wind_speed_2m:ms',
    'wind_speed_10m:ms',
    'wind_speed_50m:ms',
    'wind_speed_100m:ms'
]

epsilon = 1e-5

for f in f_logs:
    X_train[f] = np.log(X_train[f] + epsilon)
    X_test[f] = np.log(X_test[f] + epsilon)

# Converting time to datetime
# We might have done that before already
X_train['time']= pd.to_datetime(X_train['time'])
X_test['time']= pd.to_datetime(X_test['time'])

#the following two steps creates new columns to get the input for the sine & cosine columns
#creating columns indicating the hour and the month
X_train['hour'] = X_train['time'].dt.hour
X_train['month'] = X_train['time'].dt.month

X_test['hour'] = X_test['time'].dt.hour
X_test['month'] = X_test['time'].dt.month

#creating column indicating the season
def assign_season(month):
    if month in [3, 4, 5]:
        return 1  # Spring
    elif month in [6, 7, 8]:
        return 2  # Summer
    elif month in [9, 10, 11]:
        return 3  # Fall
    else:  # December, January, February
        return 4  # Winter

# X_train
X_train['season'] = X_train['month'].apply(assign_season)

X_train['hour_sine'] = np.sin(2 * np.pi * X_train['hour'] / 24)
X_train['hour_cosine'] = np.cos(2 * np.pi * X_train['hour'] / 24)

X_train['month_sine'] = np.sin(2 * np.pi * X_train['month'] / 12)
X_train['month_cosine'] = np.cos(2 * np.pi * X_train['month'] / 12)

X_train['season_sine'] = np.sin(2 * np.pi * X_train['season'] / 4)
X_train['season_cosine'] = np.cos(2 * np.pi * X_train['season'] / 4)

X_train = X_train.drop(columns = ['hour', 'month', 'season'])

# X_test
X_test['season'] = X_test['month'].apply(assign_season)

X_test['hour_sine'] = np.sin(2 * np.pi * X_test['hour'] / 24)
X_test['hour_cosine'] = np.cos(2 * np.pi * X_test['hour'] / 24)

X_test['month_sine'] = np.sin(2 * np.pi * X_test['month'] / 12)
X_test['month_cosine'] = np.cos(2 * np.pi * X_test['month'] / 12)

X_test['season_sine'] = np.sin(2 * np.pi * X_test['season'] / 4)
X_test['season_cosine'] = np.cos(2 * np.pi * X_test['season'] / 4)

X_test = X_test.drop(columns = ['hour', 'month', 'season'])

# Cyclic features
cyclical_features = ['sun_azimuth:d', 'wind_dir_2m:d', 'wind_dir_10m:d', 'wind_dir_50m:d', 'wind_dir_100m:d']
degrees = 360

for cyclical_feature in cyclical_features:
    sin_column_name = f'sin_{cyclical_feature}'
    cos_column_name = f'cos_{cyclical_feature}'
    X_train[sin_column_name] = np.sin(2 * np.pi * X_train[cyclical_feature]/2)
    X_train[cos_column_name] = np.cos(2 * np.pi * X_train[cyclical_feature]/degrees)
    X_test[sin_column_name] = np.sin(2 * np.pi * X_test[cyclical_feature]/2)
    X_test[cos_column_name] = np.cos(2 * np.pi * X_test[cyclical_feature]/degrees)

X_train = X_train.drop(columns=cyclical_features)
X_test = X_test.drop(columns=cyclical_features)

# targets = ['pv_production', 'wind_production', 'consumption']
f_minmax = [
    'hour_sine',
    'hour_cosine',
    'month_sine',
    'month_cosine',
    'season_sine',
    'season_cosine',
    'precip_1h:mm',
    'prob_precip_1h:p',
    'clear_sky_rad:W',
    'clear_sky_energy_1h:J',
    'diffuse_rad:W',
    'diffuse_rad_1h:Wh',
    'direct_rad:W',
    'direct_rad_1h:Wh',
    'global_rad:W',
    'global_rad_1h:Wh',
    'sunshine_duration_1h:min',
    'low_cloud_cover:p',
    'medium_cloud_cover:p',
    'high_cloud_cover:p',
    'total_cloud_cover:p',
    'effective_cloud_cover:p',
    'sin_sun_azimuth:d',
    'cos_sun_azimuth:d',
    'sin_wind_dir_2m:d',
    'cos_wind_dir_2m:d',
    'sin_wind_dir_10m:d',
    'cos_wind_dir_10m:d',
    'sin_wind_dir_50m:d',
    'cos_wind_dir_50m:d',
    'sin_wind_dir_100m:d',
    'cos_wind_dir_100m:d',
    'relative_humidity_2m:p',
    'relative_humidity_10m:p',
    'relative_humidity_50m:p',
    'relative_humidity_100m:p',
    'dew_point_2m:C',
    'dew_point_10m:C',
    'dew_point_50m:C',
    'dew_point_100m:C',
    'temp'
]
f_standard = ['sun_elevation:d']
f_robust = [
    't_10m:C',
    't_50m:C',
    't_100m:C',
    'wind_speed_2m:ms',
    'wind_speed_10m:ms',
    'wind_speed_50m:ms',
    'wind_speed_100m:ms'
]

f_ohe = ['precip_type:idx']

# other = ['spot_market_price']

# target
y = X_train[['pv_production', 'wind_production', 'consumption']]
# features
X_train = X_train.drop(columns=['time', 'pv_production', 'wind_production', 'consumption', 'spot_market_price'])
X_test = X_test.drop(columns=['time', 'pv_production', 'wind_production', 'consumption', 'spot_market_price'])

# Preprocessing Pipeline
minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()
cat_transformer = OneHotEncoder()
robust_scaler = RobustScaler()

preproc_basic = make_column_transformer(
    (minmax_scaler, f_minmax ),
    (standard_scaler, f_standard),
    (robust_scaler, f_robust),
    (cat_transformer, f_ohe),
    remainder='passthrough'
)
# Train X
X_train_transformed = preproc_basic.fit_transform(X_train)

# Adding Column names
X_train_transformed = pd.DataFrame(
    X_train_transformed,
    columns=preproc_basic.get_feature_names_out()
)
# Test x
X_test_transformed = preproc_basic.transform(X_test)
# Adding Column names
X_test_transformed = pd.DataFrame(
    X_test_transformed,
    columns=preproc_basic.get_feature_names_out()
)

### Creating y_train and y_test for solar production

In [2]:
# creating y_train dataframe
y_train = y.copy()['wind_production']

# Importing y_test
y_test = pd.read_csv('/Users/niki/code/Niki827/watt_squad/raw_data/test.csv')
y_test = y_test['wind_production']

### Creating X_val and y_val for solar production

In [3]:
from sklearn.model_selection import train_test_split

# Use the same function above for the validation set
X_train_transformed, X_val, y_train, y_val = train_test_split(
    X_train_transformed, y_train, test_size = 0.1, random_state = 42  # val = 10%
)

## Building an XGB Regressor

In [4]:
# install Le Wagon Version of XGBoost
#pip install xgboost==1.6.2

### Building a first version of the model

In [5]:
from xgboost import XGBRegressor

xgb_reg = XGBRegressor(max_depth=10, n_estimators=100, learning_rate = 0.1)
xgb_reg.fit(X_train_transformed, y_train,
           # evaluate the loss at each iteration
            eval_set = [(X_train_transformed, y_train), (X_val, y_val)],
            # stop iterating when eval loss increases 5 times in a row
            early_stopping_rounds = 5
           )

y_pred = xgb_reg.predict(X_test_transformed)

[0]	validation_0-rmse:40.17324	validation_1-rmse:40.38804
[1]	validation_0-rmse:37.13638	validation_1-rmse:37.89037
[2]	validation_0-rmse:34.40502	validation_1-rmse:35.74652
[3]	validation_0-rmse:31.98013	validation_1-rmse:33.79231
[4]	validation_0-rmse:29.69135	validation_1-rmse:31.99479
[5]	validation_0-rmse:27.67654	validation_1-rmse:30.48409
[6]	validation_0-rmse:25.80068	validation_1-rmse:29.17100


/Users/niki/.pyenv/versions/3.10.6/envs/watt_squad/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[7]	validation_0-rmse:24.15098	validation_1-rmse:28.01115
[8]	validation_0-rmse:22.66269	validation_1-rmse:27.01665
[9]	validation_0-rmse:21.28330	validation_1-rmse:26.09714
[10]	validation_0-rmse:20.05902	validation_1-rmse:25.41504
[11]	validation_0-rmse:18.91859	validation_1-rmse:24.77366
[12]	validation_0-rmse:17.84769	validation_1-rmse:24.26360
[13]	validation_0-rmse:16.84781	validation_1-rmse:23.80811
[14]	validation_0-rmse:15.92871	validation_1-rmse:23.37278
[15]	validation_0-rmse:15.14664	validation_1-rmse:23.10759
[16]	validation_0-rmse:14.45814	validation_1-rmse:22.79235
[17]	validation_0-rmse:13.74536	validation_1-rmse:22.46413
[18]	validation_0-rmse:13.00932	validation_1-rmse:22.15129
[19]	validation_0-rmse:12.36631	validation_1-rmse:21.98313
[20]	validation_0-rmse:11.78683	validation_1-rmse:21.81871
[21]	validation_0-rmse:11.26311	validation_1-rmse:21.66547
[22]	validation_0-rmse:10.77799	validation_1-rmse:21.47611
[23]	validation_0-rmse:10.30046	validation_1-rmse:21.29044


### Evaluating predictions of the first model on test data

In [6]:
baseline_mae = ((abs(y_train-y_train.mean())).mean())
baseline_mae

26.042127820606378

In [7]:
model_mae = ((abs(y_test-y_pred)).mean())
model_mae

20.840454673655064

### Finding the optimal parameters using GridSearch

In [8]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
import numpy as np

# Define the parameter grid for GridSearch
param_grid = {
    "max_depth": [8,9,10],            # Reduce options
    "learning_rate": [0.04, 0.05, 0.06],
    "n_estimators": [600,700,800],
    "reg_alpha": [0.01, 0.02, 0.03],
    "reg_lambda": [5, 8, 10],
    "subsample": [0.8],
    "colsample_bytree": [0.8]
}

# Initialize the model
xgb_reg = XGBRegressor(objective='reg:squarederror', eval_metric="mae", random_state=42)

# Perform the grid search
grid_search = GridSearchCV(
    estimator = xgb_reg,
    param_grid = param_grid,
    scoring = 'neg_mean_squared_error',
    cv = 3,
    verbose = 1,
    n_jobs = -1
)

# Fit the grid search
grid_search.fit(X_train_transformed, y_train)

# Get the best parameters and score
print('Best Parameters: ', grid_search.best_params_)
print('Best Score:', -grid_search.best_score_)

# Use the best model to predict
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_transformed)

Fitting 3 folds for each of 243 candidates, totalling 729 fits


KeyboardInterrupt: 

### Evaluating predictions of the grid searched model on test data

In [ ]:
baseline_mae = ((abs(y_train-y_train.mean())).mean())
baseline_mae

In [ ]:
model_mae = ((abs(y_test-y_pred)).mean())
model_mae

In [ ]:
y_train.describe()

## Feature importance and correlation matrix

### Calculating feature importance

In [ ]:
from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(X_train_transformed, y_train)

In [ ]:
import pandas as pd

importance = model.feature_importances_
feature_names = X_train_transformed.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importance})
importance_df.sort_values(by='Importance', ascending=False, inplace=True)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 12))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.gca().invert_yaxis()
plt.xlabel('Feature Importance')
plt.title('XGBoost Feature Importance')
plt.show()

In [ ]:
importance_df

### Creating a correlation matrix

In [ ]:
correlation_matrix = X_train_transformed.corr()

In [ ]:
import seaborn as sns

plt.figure(figsize=(12,10))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm')
plt.title('Feature Correlation Matrix')
plt.show()

In [ ]:
correlated_features = set()
correlation_threshold = 0.9

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i,j]) > correlation_threshold:
            correlated_features.add(correlation_matrix.columns[i])

In [ ]:
correlation_matrix[['minmaxscaler__global_rad:W']].sort_values(by='minmaxscaler__global_rad:W', ascending=False)

In [ ]:
correlated_features

In [ ]:
((abs(y_test-y_pred)).mean())

In [ ]:
((abs(y_test-y_test.mean())).mean())

## Plotting Features

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# List of the 5 most important features
important_features = [
    'minmaxscaler__global_rad:W',
    'minmaxscaler__direct_rad:W',
    'minmaxscaler__cos_sun_azimuth:d',
    'minmaxscaler__medium_cloud_cover:p',
    'minmaxscaler__prob_precip_1h:p'
]

# Create a scatterplot for each feature
for feature in important_features:
    plt.figure(figsize=(6, 4))  # Set figure size
    sns.scatterplot(x=X_train_transformed[feature], y=y_train)
    plt.title(f"Scatterplot of {feature} vs y_train")
    plt.xlabel(feature)
    plt.ylabel("y_train")
    #plt.xlim(0.4, None)  # Set the lower limit of the x-axis to 0.4
    plt.show()